<a href="https://www.kaggle.com/code/ziadhamed/data-preprocessing-and-model-selection-regression?scriptVersionId=204011392" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
! pip install -U scikit-learn

In [2]:
import pandas as pd

dataset = pd.read_csv('/kaggle/input/mydata/Data.csv')

In [3]:
dataset

,Country,Age,Salary,Purchased
0,India,34.0,92000.0,Yes
1,Sri lanka,22.0,25000.0,Yes
2,China,31.0,74000.0,Yes
3,Sri lanka,29.0,NaN,No
4,China,55.0,98000.0,Yes
5,India,24.0,30000.0,No
6,Sri lanka,28.0,40000.0,No
7,India,NaN,60000.0,No
8,China,51.0,89000.0,Yes
9,India,44.0,78000.0,Yes


In [4]:
#independent variables
X =dataset.iloc[:,:-1].values
X

array([['India', 34.0, 92000.0],
       ['Sri lanka', 22.0, 25000.0],
       ['China', 31.0, 74000.0],
       ['Sri lanka', 29.0, nan],
       ['China', 55.0, 98000.0],
       ['India', 24.0, 30000.0],
       ['Sri lanka', 28.0, 40000.0],
       ['India', nan, 60000.0],
       ['China', 51.0, 89000.0],
       ['India', 44.0, 78000.0],
       ['Sri lanka', 21.0, 20000.0],
       ['China', 25.0, 30000.0],
       ['India', 33.0, 45000.0],
       ['India', 42.0, 65000.0],
       ['Sri lanka', 33.0, 22000.0]], dtype=object)

In [5]:
#dependent variable
Y = dataset.iloc[:,3].values
Y

array(['Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes',
       'No', 'Yes', 'Yes', 'Yes', 'No'], dtype=object)

In [6]:
#handle missing values
from sklearn.impute import SimpleImputer
import numpy as np

imputer = SimpleImputer(missing_values = np.NaN, strategy = 'mean', )
imputer = imputer.fit(X[:, 1:3])
X[:,1:3]=imputer.transform(X[:,1:3])

print(X)

[['India' 34.0 92000.0]
 ['Sri lanka' 22.0 25000.0]
 ['China' 31.0 74000.0]
 ['Sri lanka' 29.0 54857.142857142855]
 ['China' 55.0 98000.0]
 ['India' 24.0 30000.0]
 ['Sri lanka' 28.0 40000.0]
 ['India' 33.714285714285715 60000.0]
 ['China' 51.0 89000.0]
 ['India' 44.0 78000.0]
 ['Sri lanka' 21.0 20000.0]
 ['China' 25.0 30000.0]
 ['India' 33.0 45000.0]
 ['India' 42.0 65000.0]
 ['Sri lanka' 33.0 22000.0]]


In [7]:
#handle categorical data
from sklearn.preprocessing import OneHotEncoder , LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

print(Y)

[1 1 1 0 1 0 0 0 1 1 0 1 1 1 0]


In [8]:
oneHotEncoder = OneHotEncoder()
X_transformed = oneHotEncoder.fit_transform(X[:,[0]]).toarray()
# Reshape the remaining columns to ensure they have two dimensions
X_remaining = X[:, 1:].reshape(X.shape[0], -1)
X_encoded = np.hstack((X_transformed , X_remaining))

print(X_encoded)

[[0.0 1.0 0.0 34.0 92000.0]
 [0.0 0.0 1.0 22.0 25000.0]
 [1.0 0.0 0.0 31.0 74000.0]
 [0.0 0.0 1.0 29.0 54857.142857142855]
 [1.0 0.0 0.0 55.0 98000.0]
 [0.0 1.0 0.0 24.0 30000.0]
 [0.0 0.0 1.0 28.0 40000.0]
 [0.0 1.0 0.0 33.714285714285715 60000.0]
 [1.0 0.0 0.0 51.0 89000.0]
 [0.0 1.0 0.0 44.0 78000.0]
 [0.0 0.0 1.0 21.0 20000.0]
 [1.0 0.0 0.0 25.0 30000.0]
 [0.0 1.0 0.0 33.0 45000.0]
 [0.0 1.0 0.0 42.0 65000.0]
 [0.0 0.0 1.0 33.0 22000.0]]


In [9]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=42)


In [10]:
#feature scaler
from sklearn.preprocessing import StandardScaler

sts = StandardScaler()
X_train[:,3:5] = sts.fit_transform(X_train[:,3:5])
X_test[:,3:5] = sts.transform(X_test[:,3:5])

print(X_train)

[[0.0 1.0 0.0 0.8182840017620521 0.5224163756674663]
 [0.0 1.0 0.0 -0.9267729949434808 -0.873860119298307]
 [1.0 0.0 0.0 1.6908125001148184 1.479863115072568]
 [1.0 0.0 0.0 -0.24813971844688468 0.8814589029443796]
 [0.0 0.0 1.0 -1.1206682167996511 -1.0733281900077032]
 [0.0 0.0 1.0 -0.054244496590714376 -1.1930090324333409]
 [1.0 0.0 0.0 2.078602943827159 1.8389056423494814]
 [0.0 1.0 0.0 0.015003796929346546 0.3229483049580702]
 [0.0 0.0 1.0 -1.2176158277277362 -1.2727962607170995]
 [0.0 1.0 0.0 -0.054244496590714376 -0.2754559071701184]
 [0.0 0.0 1.0 -0.442034940303055 0.11778114651411974]
 [0.0 0.0 1.0 -0.5389825512311401 -0.4749239778795146]]


In [11]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



models = [
    ('Linear Regression', LinearRegression()),
    ('Ridge Regression', Ridge()),
    ('Lasso Regression', Lasso()),
    ('Decision Tree', DecisionTreeRegressor()),
    ('Random Forest', RandomForestRegressor()),
    ('Support Vector Regression', SVR()),
    ('XGBoost', XGBRegressor(objective='reg:squarederror', random_state=42))
]

# Dictionary to store the MSE for each model
mse_scores = {}

# Evaluate each model
for name, model in models:
    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Predict the target variable on the test data
    y_pred = model.predict(X_test)

    # Calculate the Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)

    # Store the MSE score in the dictionary
    mse_scores[name] = mse
    r2 = r2_score(y_test, y_pred)

    print(f"{name}: MSE = {mse:.3f} \n R^2 Score = {r2:.4f}")



# Find the model with the lowest MSE
best_model_name = min(mse_scores, key=mse_scores.get)
best_mse = mse_scores[best_model_name]

print(f"\nBest Model: {best_model_name} with MSE = {best_mse:.3f}")

Linear Regression: MSE = 0.187 
 R^2 Score = 0.0000
Ridge Regression: MSE = 0.133 
 R^2 Score = 0.0000
Lasso Regression: MSE = 0.250 
 R^2 Score = 0.0000
Decision Tree: MSE = 0.333 
 R^2 Score = 0.0000
Random Forest: MSE = 0.194 
 R^2 Score = 0.0000
Support Vector Regression: MSE = 0.176 
 R^2 Score = 0.0000
XGBoost: MSE = 0.346 
 R^2 Score = 0.0000

Best Model: Ridge Regression with MSE = 0.133
